# **Disciplina CAT010** - Aplicações dos Satélites em Ciências Atmosféricas


---

## **Aula Prática 5 - Parte 1:** $\underline{Plotando\ e\ Analisando\ Relâmpagos\ Estimados\ pelo\ Satélite\ GOES-16}$
---
**OBJETIVO:** Nesta aula prática aprenderemos como trabalhar com dados de relâmpagos estimado por satélite. Para isto usaremos os dados do sensor [Geostationary Lightning Mapper (GLM)](https://space.oscar.wmo.int/instruments/view/glm) abordo do satélite [Geostationary Operational Environmental Satellite - 16 (GOES-16)](https://space.oscar.wmo.int/satellites/view/goes_16). O objetivo dessa **Parte 1** é aprender como plotar e analisar os dados a cada 20s do GLM e combiná-los com imagens de satélite.

---


**DADOS DE ENTRADA**: Arquivos de relâmpagos.

1. $\underline{Dados\ de\ Relâmpagos}$: sensor GLM do satélite GOES-16
- **Tipo do dado:** pontuais
- **Formato do dado:** arquivo NETCDF
- **Nome do arquivo:** OR_GLM-L2-LCFA_G16_s20210010306000_e20210010306204_c20210010306215.nc
- **Fonte dos dados:** FTP da [Amazon](https://noaa-goes16.s3.amazonaws.com/index.html#GLM-L2-LCFA/)

    ----------

2. $\underline{Dados\ de\ Tempertura\ de\ Brilho}$: sensor ABI do satélite GOES-16
- **Tipo do dado:** matriz
- **Formato do dado:** arquivo NETCDF
- **Nome do arquivo:** S10635346_202001231200.nc
- **Fonte dos dados:** FTP do [CPTEC/INPE](http://ftp.cptec.inpe.br/goes/goes16/retangular/ch13/)


---

**DADOS DE SAÍDA:** .
- **Tipo do dado:** Figura
- **Formato do dado:** arquivos JPG
- **Imagens geradas:**
    1. Fig_1_goes16_flash_20210101_1700.jpg
    2. Fig_2_goes16_flash_20210101_1700_recorte.jpg
    3. Fig_3_goes16_flash_20210101_1700_recorte.jpg
    4. Fig_3_animation.gif
    5. Fig_4_goes16_evento_grupo_flash_20210101_1700_recorte.jpg

---

**PROCEDIMENTO REALIZADO:** Os seguintes procedimentos são realizados nesse código:
1.   Instalação de bibliotecas
2.   Download de arquivos auxiliares
3.   Monta o google drive
4.   Define o caminho do google drive
5.   Importação das bibliotecas
6.   Define funções
7.   Conhecendo os dados de relâmpagos a cada 20 s do sensor GLM
8.   Combinando imagem de satélite com flashes: América do Sul
9.   Combinando imagem de satélite com flashes: Área com recorte
10. Faz animação das imagens combinadas de satélite e flashes: Área com recorte
11. Combinando imagem de satélite com eventos, grupos e flashes: Área com recorte


---
**OBSERVAÇÕES IMPORTANTES**:
1. Fazer mudanças, veja em: **### >>> MUDANÇAS AQUI <<< ###**
    - 1. Caminho do seu drive

---
- **Realizado por:** Enrique V. Mattos - 18/05/2022
- **Atualizado por:** Enrique V. Mattos - 11/11/2024


---


# **1° Passo:** Preparando ambiente

In [ ]:
# Installing the numpy, netcdf4, boto3 and gdal libraries
!pip install -q cartopy boto3 gdal salem rasterio pyproj geopandas descartes ultraplot

# Download dos arquivos auziliares
!wget -c https://raw.githubusercontent.com/evmpython/imagens_GOES/main/input/utilities_goes19.py
!wget -c https://github.com/evmpython/CAT010_UNIFEI_2024/raw/main/utils/ir.cpt

# Monta drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho do diretório
dir = '/content/drive/MyDrive/2-PESQUISA/0_GLM/estudos_de_caso/2025-05-27e28-FRENTEFRIA'

# Cria pasta de saída
import os
os.makedirs(f'{dir}output', exist_ok=True)

# Importa bibliotecas
import ultraplot as uplt
import time as t
import glob
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import os
from time import process_time
import calendar
#import salem

try:
    import salem
except ValueError:
    import sys
    !{sys.executable} -m pip install --upgrade numpy
    import salem

import xarray as xr
import pandas as pd

from datetime import timedelta, date, datetime
import imageio                           # Python interface to read and write a wide range of image data

import boto3                             # Amazon Web Services (AWS) SDK for Python
from botocore import UNSIGNED            # boto3 config
from botocore.config import Config       # boto3 config

from utilities_goes19 import loadCPT

from matplotlib.colors import LinearSegmentedColormap # interpolação linear para as cores dos mapas
from matplotlib import cm                             # Colormap handling utilities

import warnings
warnings.filterwarnings("ignore")

# **2° Passo:** Definindo funções

**Função que baixa os dados do GLM (GOES-19) no site de Amazon**

In [ ]:
def download_GLM(yyyymmddhhmnss, path_dest):

    # caminho dos dados do sensor GLM na Amazon
    #ftp_glm = 'https://noaa-goes19.s3.amazonaws.com/index.html#GLM-L2-LCFA/'

    os.makedirs(path_dest, exist_ok=True)

    year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%Y')

    day_of_year = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%j')

    hour = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%H')
    min = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%M')
    seg = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S').strftime('%S')

    # AMAZON repository information
    # https://noaa-goes16.s3.amazonaws.com/index.html
    bucket_name = 'noaa-goes19'

    # Initializes the S3 client
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    #-----------------------------------------------------------------------------------------------------------
    # File structure
    product_name = "GLM-L2-LCFA"
    prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}_G19_s{year}{day_of_year}{hour}{min}{seg}'

    # Seach for the file on the server
    s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

    #-----------------------------------------------------------------------------------------------------------
    # Check if there are files available
    if 'Contents' not in s3_result:

        # There are no files
        print(f'No files found for the date: {yyyymmddhhmnss}, Product-{product_name}')
        return -1

    else:

        # There are files
        for obj in s3_result['Contents']:
            key = obj['Key']

        # Print the file name
        file_name = key.split('/')[-1].split('.')[0]

        # Download the file
        if os.path.exists(f'{path_dest}/{file_name}.nc'):
            print(f'File {path_dest}/{file_name}.nc exists')
        else:
            print(f'Downloading file {path_dest}/{file_name}.nc')
            s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')

    return f'{file_name}'

**Função que baixa as imagens do ABI no ftp do CPTEC**

In [ ]:
def download_ABI_CPTEC(ano, mes, dia, hor, min, canal):

    print('.... Baixando dado ===>', 'DATA=', ano, mes, dia, hor, min, '/', 'CANAL=', canal)

    # caminho do ftp do cptec: https://ftp.cptec.inpe.br/goes/goes19/retangular/ch13/
    ftp_cptec = 'http://ftp.cptec.inpe.br/goes/'

    # define codigo do produto
    if canal == 'ch02':
        codigo_produto = 'S10635334'
    elif canal == 'ch08':
        codigo_produto = 'S10635340'
    elif canal == 'ch13':
        codigo_produto = 'S10165545'

    # nome do arquivo
    file_ir =  f'{ftp_cptec}goes19/retangular/{canal}/{ano}/{mes}/{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc'

    # baixando a imagem de satélite
    !wget -c {file_ir}

    # local da imagem que foi baixada
    file_ir_download = f'/content/{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc'

    return file_ir_download

**Função que baixa e acumula flashes no intervalo temporal da imagem**

In [ ]:
def download_e_acumula_GLM(ano, mes, dia, hor, min):

    # define data inicial e final para acumular os relâmpagos
    date_ini = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)))
    date_end = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)) + timedelta(minutes=10)) # soma 10 minutos na data inicial

    # inicializa os arrays de latitude e longitude
    lats_event, lons_event = np.array([]), np.array([])
    lats_group, lons_group = np.array([]), np.array([])
    lats_flash, lons_flash = np.array([]), np.array([])

    # loop da acumulação de relâmpagos
    while (date_ini < date_end):

        # estrutura da data
        yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

        # baixando GLM
        file_glm = download_GLM(yyyymmddhhmnss, '/content')

        # leitura GLM
        ds = xr.open_dataset(f'/content/{file_glm}.nc')

        # apenda as lats/lons dos eventos, grupos e flashes
        lats_event = np.append(lats_event, ds['event_lat'])
        lons_event = np.append(lons_event, ds['event_lon'])

        lats_group = np.append(lats_group, ds['group_lat'])
        lons_group = np.append(lons_group, ds['group_lon'])

        lats_flash = np.append(lats_flash, ds['flash_lat'])
        lons_flash = np.append(lons_flash, ds['flash_lon'])

        # incrementa 20s no tempo
        date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))

    return lats_event, lons_event, lats_group, lons_group, lats_flash, lons_flash

# **PARTE 1)**: Combinando imagem de satélite com flashes: América do Sul

##**a) Definindo data/horário e canal da imagem**

In [ ]:
# define data/horário e canal
ano, mes, dia, hor, min, canal = '2025', '05', '27', '23', '40', 'ch13'

In [ ]:
# baixa imagem de satélite do FTP do CPTEC
file_ir_download = download_ABI_CPTEC(ano, mes, dia, hor, min, canal)

##**b) Acumula os flashes**

In [ ]:
# define data a INICIAL para acumular os relâmpagos
date_ini = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)))

# define data FINAL para acumular os relâmpagos
date_end = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)) + timedelta(minutes=10)) # soma 10 minutos na data inicial

In [ ]:
# mostra na tela as datas
print('Tipo da variável=', type(date_ini))
print('Tipo da variável=', type(date_end))
print('Data Inicial=', date_ini)
print('Data Final=', date_end)

In [ ]:
# inicializa os arrays de latitude e longitude
lats_event, lons_event = np.array([]), np.array([])
lats_group, lons_group = np.array([]), np.array([])
lats_flash, lons_flash = np.array([]), np.array([])

# loop da acumulação de relâmpagos
while (date_ini < date_end):

    # mostra na tela a data de processamento atual
    print('PROCESSANDO DATA....', date_ini)

    # estrutura da data
    yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # baixando GLM
    file_glm = download_GLM(yyyymmddhhmnss, '/content')

    # leitura GLM
    ds = xr.open_dataset(f'/content/{file_glm}.nc')

    # apenda as lats/lons dos eventos, grupos e flashes
    lats_event = np.append(lats_event, ds['event_lat'])
    lons_event = np.append(lons_event, ds['event_lon'])

    lats_group = np.append(lats_group, ds['group_lat'])
    lons_group = np.append(lons_group, ds['group_lon'])

    lats_flash = np.append(lats_flash, ds['flash_lat'])
    lons_flash = np.append(lons_flash, ds['flash_lon'])

    # fecha o arquivo
    ds.close()

    # incrementa 20s no tempo
    date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    print('\n')

##**c) Plota figura**

In [ ]:
%%time
# Leitura do arquivo
imagem = xr.open_dataset(file_ir_download)

# Limites da imagem
latmin, latmax, lonmin, lonmax = float(imagem['lat'][0]), float(imagem['lat'][-1]), float(imagem['lon'][0]), float(imagem['lon'][-1])

# Divide por 100 e transforma para Celsius
imagem = (imagem['Band1']/100.)-273.15

# Inverte a matriz
imagem = np.flipud(imagem)

# moldura da figura
fig, ax = uplt.subplots(axwidth=5, axheight=5, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=True, borders=True, innerborders=False,
          labels=True,
          latlines=20, lonlines=20,
          latlim=(latmin,latmax), lonlim=(lonmin,lonmax),
          title=f'Temperatura (ABI, 10.35 µm) + Flashes (GLM): {ano}-{mes}-{dia} às {hor}:{min} UTC',
          small='20px', large='15px')

# carrega tabela de cores
cpt_ir = loadCPT('/content/ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

# plota mapa
map1 = ax.imshow(imagem,
                 cmap=cmap_ir,
                 extent=[lonmin, lonmax, latmin, latmax],
                 levels=uplt.arange(-103.0, 105, 1.0),
                 alpha=0.5, zorder=1)

#----------------------------------------------#
#             Plota os flashes
#----------------------------------------------#
# coloca os flashes num dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# seleciona os flashes da região
df_flash_filtered = df[ (df['lon'] > lonmin) & (df['lon'] < lonmax) & (df['lat'] > latmin) & (df['lat'] < latmax)]

# transforma de dataframe para array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

# plota flashes
ax.scatter(lons_flash_filtered, lats_flash_filtered,
           transform=ccrs.PlateCarree(),
           marker='x', s=0.5, facecolor='black', edgecolor='black',
           linewidth=1, alpha=0.8, zorder=2,
           label=f'Flashes={len(lats_flash_filtered)}')
#----------------------------------------------#

# plota contornos dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/CAT010_UNIFEI_2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.8, zorder=2)

# plota barra de cores da figura
fig.colorbar(map1, loc='r', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=13, labelsize=15, space=0.4) #, length=0.99

# adiciona legenda
ax.legend(loc='lr', ncols=1, frameon=True)

# salva figura
fig.save(f'{dir}/output/Fig_1_goes19_flash_{ano}{mes}{dia}_{hor}{min}.jpg', dpi=300, bbox_inches='tight')

# **PARTE 2)**: Combinando imagem de satélite com flashes: Área com recorte

In [ ]:
# define a área de recorte
lonmin_rec, lonmax_rec, latmin_rec, latmax_rec = -65, -45, -35, -15

# Leitura do arquivo para a área de recorte
imagem = xr.open_dataset(file_ir_download).sel(lon=slice(lonmin_rec, lonmax_rec), lat=slice(latmin_rec, latmax_rec))

# Limites da imagem
latmin, latmax, lonmin, lonmax = float(imagem['lat'][0]), float(imagem['lat'][-1]), float(imagem['lon'][0]), float(imagem['lon'][-1])

# Divide por 100 e transforma para Celsius
imagem = (imagem['Band1']/100.)-273.15

# Inverte a matriz
imagem = np.flipud(imagem)

# moldura da figura
fig, ax = uplt.subplots(axwidth=5, axheight=5, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=False, borders=True, innerborders=False,
          labels=True,
          latlines=5, lonlines=5,
          latlim=(latmin_rec,latmax_rec), lonlim=(lonmin_rec,lonmax_rec),
          title=f'Temperatura (ABI, 10.35 µm) + Flashes (GLM): {ano}-{mes}-{dia} às {hor}:{min} UTC',
          small='20px', large='15px')

# carrega tabela de cores
cpt_ir = loadCPT('/content/ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

# plota mapa
map1 = ax.imshow(imagem,
                 cmap=cmap_ir,
                 extent=[lonmin_rec, lonmax_rec, latmin_rec, latmax_rec],
                 levels=uplt.arange(-103.0, 105, 1.0),
                 alpha=0.8, zorder=1)

#----------------------------------------------#
#            Plota os flashes
#----------------------------------------------#
# coloca os flashes num dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# seleciona os flashes da região
df_flash_filtered = df[ (df['lon'] > lonmin_rec) & (df['lon'] < lonmax_rec) & (df['lat'] > latmin_rec) & (df['lat'] < latmax_rec)]

# transforma de dataframe para array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

# plota flashes
ax.scatter(lons_flash_filtered, lats_flash_filtered,
           transform=ccrs.PlateCarree(),
           marker='o', s=20, facecolor='white', edgecolor='black',
           linewidth=1, alpha=0.8, zorder=2,
           label=f'Flashes={len(lats_flash_filtered)}')
#----------------------------------------------#

# plota contornos dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/CAT010_UNIFEI_2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0, zorder=2)

# plota barra de cores da figura
fig.colorbar(map1, loc='r', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=13, labelsize=15, space=0.3, length=0.71)

# adiciona legenda
ax.legend(loc='lr', ncols=1, frameon=True)

# salva figura
fig.save(f'{dir}/output/Fig_2_goes19_flash_{ano}{mes}{dia}_{hor}{min}_recorte.jpg', dpi=300, bbox_inches='tight')

# **PARTE 3)**: Faz animação das imagens combinadas de satélite e flashes: Área com recorte

##**a) Define o período de imagens de satélite**

In [ ]:
# data INICIAL
anoi, mesi, diai, hori, mini = 2021, 1, 1, 17, 0 # ano, mês, dia, hora e minuto inicial do período

# data FINAL
anof, mesf, diaf, horf, minf = 2021, 1, 1, 18, 30 # ano, mês, dia, hora e minuto final do período

In [ ]:
# define a área de recorte
lonmin_rec, lonmax_rec, latmin_rec, latmax_rec = -42, -35, -30, -25

##**b) Download das imagens de satélite**

In [ ]:
# Inicia o contador de tempo
start_time = t.time()

# Data desejada
date_in = datetime(anoi, mesi, diai, hori, mini)  # 2022-02-02 00:00:00
date_ini = date_in.strftime('%Y%m%d%H%M') # 20220202

date_en = datetime(anof, mesf, diaf, horf, minf)
date_end = date_en.strftime('%Y%m%d%H%M')

print('.... Processando Data ...:', date_ini, date_end, '\n')

# Download dos dados
files_imagens = []
for file in pd.date_range(date_ini,
                          date_end,
                          freq='10min'):

    # Extrai ano e mes
    data = file.strftime('%Y%m%d%H%M')
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
    hor = file.strftime('%H')
    min = file.strftime('%M')

    # baixa imagem de satélite
    file_ir_download = download_ABI_CPTEC(ano, mes, dia, hor, min, canal)

    # salva o nomes das imagens baixadas
    files_imagens.append(file_ir_download)

# Contabiliza o tempo para baixar os dados
print('\nTempo de download dos dados:', round((t.time() - start_time),2), 'segundos.')

In [ ]:
# mostra os nomes das imagens de satélite que foram baixadas
files_imagens

##**c) Plotando as imagens de satélite + relâmpagos**

In [ ]:
# Loop nas imagens baixadas
for file in files_imagens:

    # extrai a data e horário da imagem
    basename = os.path.basename(os.path.splitext(file)[0]) # S10635334_202101011700.nc
    ano, mes, dia, hor, min = basename[10:14], basename[14:16], basename[16:18], basename[18:20], basename[20:22]

    print('---------------------------------------------------------------------')
    print('PROCESSANDO IMAGEM=', basename)
    print('---------------------------------------------------------------------')

    # Leitura do arquivo
    imagem = xr.open_dataset(file).sel(lon=slice(lonmin_rec, lonmax_rec), lat=slice(latmin_rec, latmax_rec))

    # Divide por 100 e transforma para Celsius
    imagem = (imagem['Band1']/100.)-273.15

    # Inverte a matriz
    imagem = np.flipud(imagem)

    # moldura da figura
    fig, ax = plot.subplots(axwidth=5, axheight=5, tight=True, proj='pcarree')

    # define formato da figura
    ax.format(coast=False, borders=False, innerborders=False,
              labels=True,
              latlines=2, lonlines=2,
              latlim=(latmin,latmax), lonlim=(lonmin,lonmax),
              title=f'Temperatura (ABI, 10.35 µm) + Flashes (GLM): {ano}-{mes}-{dia} às {hor}:{min} UTC',
              small='20px', large='15px')

    # carrega tabela de cores
    cpt_ir = loadCPT('/content/ir.cpt')
    cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

    # plota mapa
    map1 = ax.imshow(imagem,
                     cmap=cmap_ir,
                     extent=[lonmin_rec, lonmax_rec, latmin_rec, latmax_rec],
                     levels=plot.arange(-103.0, 105, 1.0),
                     alpha=0.8, zorder=1)

    # baixa dados do GLM
    lats_event, lons_event, lats_group, lons_group, lats_flash, lons_flash = download_e_acumula_GLM(ano, mes, dia, hor, min)

    #----------------------------------------------#
    #          Plotas os flashes
    #----------------------------------------------#
    # coloca os flashes num dataframe
    data_flash = {'lat': lats_flash, 'lon': lons_flash}
    df = pd.DataFrame(data_flash)

    # seleciona os flashes da região
    df_flash_filtered = df[ (df['lon'] > lonmin_rec) & (df['lon'] < lonmax_rec) & (df['lat'] > latmin_rec) & (df['lat'] < latmax_rec)]

    # transforma de dataframe para array
    lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

    # plota flashes
    ax.scatter(lons_flash_filtered, lats_flash_filtered,
               transform=ccrs.PlateCarree(),
               marker='o', s=20, facecolor='white', edgecolor='black',
               linewidth=1, alpha=0.8, zorder=2,
               label=f'Flashes={len(lats_flash_filtered)}')
    #----------------------------------------------#

    # plota contornos dos Estados
    shapefile = list(shpreader.Reader('https://github.com/evmpython/CAT010_UNIFEI_2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.8, zorder=2)

    # plota barra de cores da figura
    fig.colorbar(map1, loc='r', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=13, labelsize=15, space=0.3, length=0.71)

    # adiciona legenda
    ax.legend(loc='lr', ncols=1, frameon=True)

    # salva figura
    fig.save(f'{dir}output/Fig_3_goes16_flash_{ano}{mes}{dia}_{hor}{min}_recorte.jpg', dpi=300, bbox_inches='tight')

##**Faz animação**

In [ ]:
# Images we want to include in the GIF
files = sorted(glob.glob(f'{dir}output/Fig_3_goes16_flash_*.jpg'))
files

In [ ]:
%%time
# cria o GIF
images = []
for file in files:
    images.append(imageio.imread(file))

# salva o GIF
imageio.mimsave(f'{dir}output/Fig_3_animation.gif',
                images,
                duration=500,
                loop=0)

# mostrando a animação na tela
print("\nAbrindo o GIF..\n")

from IPython.display import Image

Image(open(f'{dir}output/Fig_3_animation.gif','rb').read(), width=600)